In [ ]:
import pandas as pd
from nltk.tokenize import RegexpTokenizer
from collections import Counter

In [ ]:
data = pd.read_csv('https://raw.githubusercontent.com/dnrocha1/information_retrieval/master/lab02/data/results.csv')

In [ ]:
#pre-process

documents = data['text'].apply(lambda x: x.lower())

regex = RegexpTokenizer(r'\b[A-zÀ-ú-\'\d]{3,}')
# tokens = regex.tokenize(texts)

In [ ]:
def build_index(documents=documents):
  inverted_list = {}

  n_doc = 0
  for document in documents:
    n_doc += 1
    token = regex.tokenize(document)
    counter = list(Counter(token).items())
    for elem in counter:
      key = elem[0]
      freq = elem[1]
      if key in inverted_list.keys():
        if n_doc not in inverted_list[key][0]:
          inverted_list[key].append((n_doc,freq))
      else:
        inverted_list[key] = [(n_doc,freq)]
  
  return inverted_list

build_index()